In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import cv2
import sscCdi
import scipy

from skimage import data
from skimage.registration import phase_cross_correlation
from skimage.registration._phase_cross_correlation import _upsampled_dft

%matplotlib widget

In [2]:
def alignment_variance_field(data,upsampling_factor=10,return_common_valid_region=False):
    
    neighbor_shifts = np.empty((data.shape[0],2))
    
    print('Finding shift between neighboor slices...')
    for i in range(0,data.shape[0]-1):

        if i%10 == 0: print(i)
        
        local_variance1 = calculate_local_variance_field(data[i])
        local_variance2 = calculate_local_variance_field(data[i+1])
        
        shift, error, diffphase = phase_cross_correlation(local_variance1, local_variance2, upsample_factor=upsampling_factor)

        neighbor_shifts[i][0] = shift[0]
        neighbor_shifts[i][1] = shift[1]
        
    total_shift = np.cumsum(neighbor_shifts,axis=0)
    aligned_volume = np.zeros_like(data)
    aligned_volume[0] = data[0]
    
    print('Shifting slices...')
    for i in range(0,data.shape[0]-1):
        if i%100 == 0: print(i)
        aligned_volume[i+1] = scipy.ndimage.shift(data[i+1],total_shift[i])
    
    if return_common_valid_region:
        print('Finding common useful region for whole volume...')
        masked_volume = np.where(aligned_volume==0,0,1)
        product = np.prod(np.abs(masked_volume),axis=0)
        where_null = np.where(np.abs(product) == 0,0,1)

        # plt.figure()
        # plt.imshow(np.abs(aligned_volume[-1]))

        aligned_volume[:] = np.where(where_null==1,aligned_volume,0) 

    print('Done')
    return aligned_volume, neighbor_shifts

def shift_2d_replace(data, dx, dy, constant=False):
    """
    Shifts the array in two dimensions while setting rolled values to constant
    :param data: The 2d numpy array to be shifted
    :param dx: The shift in x
    :param dy: The shift in y
    :param constant: The constant to replace rolled values with
    :return: The shifted array with "constant" where roll occurs
    """
    shifted_data = np.roll(data, dx, axis=1)
    if dx < 0:
        shifted_data[:, dx:] = constant
    elif dx > 0:
        shifted_data[:, 0:dx] = constant

    shifted_data = np.roll(shifted_data, dy, axis=0)
    if dy < 0:
        shifted_data[dy:, :] = constant
    elif dy > 0:
        shifted_data[0:dy, :] = constant
    return shifted_data


def get_gaussian_pyramid(frame,layers=2):

    from skimage.transform import pyramid_gaussian

    img_real = np.real(frame)
    img_imag = np.imag(frame)

    pyramid_real = tuple(pyramid_gaussian(img_real,downscale=2,max_layer=layers-1))
    pyramid_imag = tuple(pyramid_gaussian(img_imag,downscale=2,max_layer=layers-1))
    
    pyramid = []
    for R, I in zip(pyramid_real,pyramid_imag):
        pyramid.append(R+1j*I)

    return pyramid

def calculate_local_variance_field(matrix):
    
    gradient = np.gradient(matrix)
    del_x = gradient[1]
    del_y = gradient[0]
        
    return np.sqrt(np.abs(del_x)**2 + np.abs(del_y)**2)


def remove_black_borders(aligned_volume):

    not_null = np.argwhere(np.abs(aligned_volume[0]))

    # Bounding box of non-black pixels.
    x0, y0 = not_null.min(axis=0)
    x1, y1 = not_null.max(axis=0) + 1   # slices are exclusive at the top

    # Get the contents of the bounding box.
    cropped_aligned_volume = aligned_volume[:,x0:x1, y0:y1]

    return cropped_aligned_volume

In [ ]:
# path = "/ibira/lnls/labs/tepui/home/yuri.tonin/00000000/proc/recons/glass_sphere/temp/2023-05-16-15h38m_glass_sphere_phase_cropped_sinogram.npy"
# data = np.load(path)[0:10]

path = "/ibira/lnls/beamlines/caterete/proposals/20221798/proc/recons/sorgoBMR08_mem1_FAA_YuriTest/YuriTest_sorgoBMR08_mem1_FAA.hdf5"
data = h5py.File(path,'r')['recon/object'][()]
angles = h5py.File(path,'r')['recon/angles'][()][:,0::2]

# mean = np.mean(data,axis=0)
# plt.figure()
# plt.imshow(np.abs(mean))

In [ ]:
data=data[:,1345:3040,1280:2980]

In [ ]:
mean = np.mean(data,axis=0)
plt.figure()
plt.imshow(np.abs(mean))

In [ ]:
from sscCdi import sort_angles
from sscCdi import reorder_slices_low_to_high_angle

sorted_angles = sort_angles(angles) # input colums with frame number and angle in rad
data = reorder_slices_low_to_high_angle(data, sorted_angles)

In [ ]:
aligned_volume, neighbor_shifts = alignment_variance_field(data,return_common_valid_region=True)

In [ ]:
sscCdi.misc.deploy_visualizer(np.abs(data),title='sample',cmap='gray')

In [ ]:
sscCdi.misc.deploy_visualizer(np.angle(aligned_volume),title='sample',cmap='gray')

In [ ]:
cropped_aligned_volume = remove_black_borders(aligned_volume)

cropped_aligned_volume = np.angle(cropped_aligned_volume)

In [ ]:
sscCdi.misc.deploy_visualizer(cropped_aligned_volume,title='sample',cmap='gray')

In [ ]:
from sscCdi import unwrap_in_parallel, equalize_frames_parallel
cropped_aligned_volume = unwrap_in_parallel(cropped_aligned_volume)

In [ ]:
dic={}
dic["bad_frames_before_equalization"] = [0,1,508,509]
dic["equalize_gradient"] = True
dic["equalize_global_offset"] = False
dic["equalize_invert"] = True
dic["equalize_local_offset"] = [1400,1500,1200,1400]
dic["equalize_outliers"] = 0

equalized_sinogram = equalize_frames_parallel(cropped_aligned_volume,dic["equalize_invert"],dic["equalize_gradient"],dic["equalize_outliers"],dic["equalize_global_offset"], dic["equalize_local_offset"])
sscCdi.misc.deploy_visualizer(equalized_sinogram,title='sample',cmap='gray')

In [ ]:
# path1 = '/ibira/lnls/beamlines/caterete/proposals/20221798/proc/recons/sorgoBMR08_mem1_FAA_01/temp/2023-07-18-12h48m_sorgoBMR08_mem1_FAA_01_cross_correlation_alignment.npy'
# np.save(path1,equalized_sinogram)

In [ ]:
sscCdi.misc.deploy_visualizer(equalized_sinogram,title='sample',cmap='gray',axis=1,limits=(0,9.5))

In [ ]:
sscCdi.misc.deploy_visualizer(equalized_sinogram,title='sample',cmap='gray',axis=1,limits=(0,9.5))

In [ ]:
path = '/ibira/lnls/beamlines/caterete/proposals/20221798/proc/recons/sorgoBMR08_mem1_FAA_01/temp/2023-07-18-12h48m_sorgoBMR08_mem1_FAA_01_phase_wiggle_sinogram.npy'

only_wiggle_alignment = np.load(path)

sscCdi.misc.deploy_visualizer(only_wiggle_alignment,title='sample',cmap='gray',axis=1)

In [ ]:
from sscCdi import wiggle

dic = {}
dic["CPUs"] = 32
dic["project_angles_to_regular_grid"] = False
dic["bad_frames_before_wiggle"] = [] 
dic["wiggle_reference_frame"] = 100

aligned_sinogram, wiggle_cmas = wiggle(dic,equalized_sinogram)
# for frame in [28,30,45,49,65,66,91,113]:
    # aligned_sinogram[frame] = np.zeros(equalized_sinogram[0].shape)

sscCdi.misc.deploy_visualizer(aligned_sinogram,title='sample',cmap='gray',axis=1)

In [ ]:
# path2 = '/ibira/lnls/beamlines/caterete/proposals/20221798/proc/recons/sorgoBMR08_mem1_FAA_01/temp/2023-07-18-12h48m_sorgoBMR08_mem1_FAA_01_cross_correlation_and_wiggle.npy'
# np.save(path2,aligned_sinogram)